In [1]:
!pip install efficientnet_pytorch

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-cp37-none-any.whl size=16443 sha256=5a28bcd6ce32f7f4b27a45e5f6aabd85989986aafd239dbbbab89149f7336013
  Stored in directory: /root/.cache/pip/wheels/84/27/aa/c46d23c4e8cc72d41283862b1437e0b3ad318417e8ed7d5921
Successfully built efficientnet-pytorch


In [2]:
import torch
import torchvision
import torchvision.transforms as transforms

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# CUDA 기기가 존재한다면, 아래 코드가 CUDA 장치를 출력합니다:

print(device)

cuda:0


In [4]:
from efficientnet_pytorch import EfficientNet
import gc
gc.collect()
torch.cuda.empty_cache()

In [5]:
transform = transforms.Compose([
    transforms.Resize(224, interpolation=2),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))])
    #https://github.com/kuangliu/pytorch-cifar/issues/19 참고
'''
여러 transform들을 Compose로 구성
여기서는 ToTensor와 Normalize를 Compose
ToTensor: numpy 배열을 pytorch tensor로 변형
Normalize: 평균과 표준편차 사용해서 이미지 정규화
Normalize(mean, std, inplace=False)
inplace=True로 하면 내용이 바뀜, False면 따로 변수에 넣어줘야 함
'''

batch_size = 64
# batch size를 4로 고정

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
'''
root  : root directory of dataset
train : if True, 데이터셋을 training set에서 만든다. False면 test set에서
download: True면 다운로드 해서 root directory에 넣음
transform: optional, PIL image를 받아서 transformed 버전으로 만드는 function(transform)
'''

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
'''
trainset: dataset
shuffle: True면 매 epoch마다 shuffle함
num_workers: data loading 과정에서 얼마나 많은 subprocess들을 사용할지
'''

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)


testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)
dataloaders = {'train' : trainloader, 'test' : testloader }
dataset_sizes = { 'train' : len(trainset) , 'test' : len(testset) }

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:281: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  "Argument interpolation should be of type InterpolationMode instead of int. "



Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [7]:
import torch.nn as nn
import torch.nn.functional as F

net = EfficientNet.from_pretrained('efficientnet-b0',num_classes = 10)
net = net.to(device)


Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b0-355c32eb.pth



Loaded pretrained weights for efficientnet-b0


In [8]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(net.parameters(), lr=0.001)


In [9]:
import time
gc.collect()
torch.cuda.empty_cache()

In [12]:
def train(net,criterion,optimizer,num_epochs=4):
  start = time.time()
  #학습
  train_loss, train_acc, test_loss, test_acc = [], [], [], []
  for epoch in range(num_epochs):
    for phase in ['train','test']:
      running_loss = 0.0
      running_corrects = 0
      num_cnt = 0
      if phase == 'train':
        net.train()
      else:
        net.eval()
      for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        with torch.set_grad_enabled(phase == 'train'):
          outputs = net(inputs).cuda()
          _, preds = torch.max(outputs, 1) 
          loss = criterion(outputs, labels).cuda()
          # backward + optimize only if in training phase
          if phase == 'train': # training 모드에서는 weight를 update한다.
              loss.backward()
              optimizer.step()
        outputs = net(inputs)
        running_loss += loss.item()
        preds = preds.cuda()
        running_corrects += torch.sum(preds == labels.data)
        num_cnt += len(labels)
      epoch_loss = running_loss / num_cnt
      epoch_acc = running_corrects.double().item() / num_cnt
      if phase == 'train':
          train_loss.append(epoch_loss)
          train_acc.append(epoch_acc)
      else:
          test_loss.append(epoch_loss)
          test_acc.append(epoch_acc)
      print("loss: ",epoch_loss,"accuracy: ",epoch_acc)

  print('Finished Training')
  print(time.time()-start)
  return train_loss, train_acc, test_loss, test_acc


In [13]:
train_loss, train_acc, test_loss, test_acc = train(net,criterion,optimizer,4)

loss:  0.0028808720944076775 accuracy:  0.93816
loss:  0.002537873363830149 accuracy:  0.94356
loss:  0.00229962644379586 accuracy:  0.95012
loss:  0.0013865305320359767 accuracy:  0.97012
loss:  0.0018980429113656283 accuracy:  0.95944
loss:  0.0014467252270504833 accuracy:  0.96838
loss:  0.0015793115111347289 accuracy:  0.9648
loss:  0.0017671724853478372 accuracy:  0.96362


KeyboardInterrupt: ignored

Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe


In [ ]:
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)
#모델 저장

In [ ]:
# 각 분류(class)에 대한 예측값 계산을 위해 준비
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# 변화도는 여전히 필요하지 않습니다
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = net(images)
        _, predictions = torch.max(outputs, 1)
        # 각 분류별로 올바른 예측 수를 모읍니다
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# 각 분류별 정확도(accuracy)를 출력합니다
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print("Accuracy for class {:5s} is: {:.1f} %".format(classname,
                                                   accuracy))